In [1]:
import sys
print(sys.version) # broai supports python3.11

3.11.12 (main, Apr  9 2025, 04:04:00) [Clang 20.1.0 ]


# Start

In [2]:
%load_ext autoreload
%autoreload 2

# DuckStore

## Relational

In [3]:
from broai.duckdb_management.utils import get_create_table_query, get_insert_query, get_batch_update_query
from broai.duckdb_management.interface import DuckStoreInterface
from broai.duckdb_management.utils import DataTypeConversion
from broai.experiments.huggingface_embedding import EmbeddingDimension

/home/ec2-user/SageMaker/broai/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
converted_text = DataTypeConversion.convert_single_quote(text="My name's Jake.")
converted_text

'My name<|single_quote|>s Jake.'

In [5]:
reversed_text = DataTypeConversion.reverse_single_quote(text=converted_text)
reversed_text

"My name's Jake."

In [6]:
schemas = {
    "doc_id": "VARCHAR",
    "content": "VARCHAR",
    "data": "JSON"
}

sm = DuckStoreInterface(db_name="./duckmemory.db", table="sessionmemory", schemas=schemas)

In [7]:
sm.create_table()

In [8]:
sm.sql_df(query="SELECT * FROM sessionmemory;")

,doc_id,content,data


In [9]:
sm.show_schemas()

{'doc_id': 'VARCHAR', 'content': 'VARCHAR', 'data': 'JSON'}

In [10]:
_data = [
    ["0", "a"],
    ["1", "b"]
]
data = ", ".join([f"('{d[0]}', '{d[1]}')" for d in _data])
sm.add(fields=["doc_id", "content"], data=data)
sm.read(fields=["*"])

,doc_id,content,data
0,0,a,None
1,1,b,None


In [11]:
_data = [
    ["0", "aa"],
    ["1", "bb"]
]
data = ", ".join([f"('{d[0]}', '{d[1]}')" for d in _data])
sm.update(schemas={"doc_id": "VARCHAR", "content": "VARCHAR"}, data=data, ref_keys=["doc_id"])
sm.read()

,doc_id,content,data
0,0,aa,None
1,1,bb,None


In [12]:
sm.delete(where_condition="WHERE doc_id IN ('1', '2')")
sm.read()

,doc_id,content,data
0,0,aa,None


In [13]:
sm.delete_table()

In [14]:
sm.drop_table()

In [15]:
sm.remove_database(confirm=f"remove {sm.db_name}")

## Vector Search: not implement yet

In [16]:
schemas = {
    "id": "VARCHAR",
    "vectors": "FLOAT[3]"
}
vector_db = DuckStoreInterface(db_name="./duckmemory.db", table="vectors", schemas=schemas)
vector_db.create_table()

In [17]:
vector_db.read()

,id,vectors


In [18]:
vectors = [
    [0.0001,0.0001,0.0001],
    [0,.1,0],
    [.15,0,.10],
]
data = ", ".join(f"('{i}', {v})" for i, v in zip(["a", "b", "c"], vectors))
vector_db.add(fields=["id", "vectors"], data=data)
vector_db.read()

,id,vectors
0,a,"[1e-04, 1e-04, 1e-04]"
1,b,"[0.0, 0.1, 0.0]"
2,c,"[0.15, 0.0, 0.1]"


In [19]:
from typing import List
import duckdb

def vector_search(db:DuckStoreInterface, field:str, vector:List[float], embedding_size:int, limit=2):
    if len(vector) != embedding_size:
        raise Exception(f"vector must be of size, {embedding_size}. Instead got {len(vector)}")
    db_name = db.db_name
    table = db.table
    query = f"""SELECT *, array_cosine_similarity({field}, $searchVector::FLOAT[3]) AS score from {table} ORDER BY score DESC LIMIT {limit};"""
    with duckdb.connect(db_name) as con:
        df = con.sql(query, params=dict(searchVector=vector)).to_df()
    return df

In [20]:
vector = [0.0001,0.0001,0.0001]

vector_search(vector_db, field="vectors", vector=vector, embedding_size=3)

,id,vectors,score
0,a,"[1e-04, 1e-04, 1e-04]",1.000000
1,c,"[0.15, 0.0, 0.1]",0.800641


In [21]:
vector_db.delete_table()

## Fulltext Search: not implement yet

In [22]:
schemas = {
    "id": "VARCHAR",
    "target_field": "VARCHAR"
}

fulltext_db = DuckStoreInterface(db_name="./duckmemory.db", table="fulltext", schemas=schemas)
fulltext_db.create_table()

In [23]:
ids = ["a", "b", "c", "d"]
targets = ["pandas", "pandee", "pandos", "PANDAS"]
data = ", ".join([f"('{i}', '{t}')" for i, t in zip(ids, targets)])

fulltext_db.add(fields=['id', 'target_field'], data=data)
fulltext_db.read()

,id,target_field
0,a,pandas
1,b,pandee
2,c,pandos
3,d,PANDAS


In [24]:
def create_fts_index(db:DuckStoreInterface, fields:List[str]):
    db_name = db.db_name
    table = db.table
    params = ", ".join([f"'{f}'" for f in fields])
    query=f"""
    INSTALL fts;
    LOAD fts;
    PRAGMA create_fts_index(
        '{table}', {params}, overwrite=1
    );
    """.strip()
    with duckdb.connect(db_name) as con:
        con.sql(query)

In [25]:
create_fts_index(db=fulltext_db, fields=["id", "target_field"])

In [26]:
fulltext_db.read()

,id,target_field
0,a,pandas
1,b,pandee
2,c,pandos
3,d,PANDAS


In [27]:
def fts_bm25(db:DuckStoreInterface, search_term:str, id_field, search_field:str):
    db_name = db.db_name
    table = db.table
    query = f"""\
    SELECT *
    FROM (
        SELECT *, fts_main_{table}.match_bm25(
            {id_field},
            '{search_term}',
            fields := '{search_field}'
        ) AS score
        FROM {table}
    ) sq
    ORDER BY score DESC;
    """
    with duckdb.connect(db_name) as con:
        df = con.sql(query).to_df()
    return df

In [28]:
fts_bm25(db=fulltext_db, search_term="pand", id_field="id", search_field="target_field")

,id,target_field,score
0,a,pandas,NaN
1,b,pandee,NaN
2,c,pandos,NaN
3,d,PANDAS,NaN


In [29]:
fulltext_db.delete_table()